In [1]:
!pip install max==25.4.0 --index-url https://dl.modular.com/public/nightly/python/simple/

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.0/285.0 MB 4.3 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/modular/mojo-gpu-puzzles

Cloning into 'mojo-gpu-puzzles'...
remote: Enumerating objects: 6332, done.
remote: Counting objects: 100% (481/481), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 6332 (delta 449), reused 416 (delta 416), pack-reused 5851 (from 3)
Receiving objects: 100% (6332/6332), 148.64 MiB | 23.40 MiB/s, done.
Resolving deltas: 100% (3923/3923), done.


In [3]:
!curl -fsSL https://astral.sh/uv/install.sh | sh

downloading uv 0.8.14 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [4]:
import max.support.notebook

In [5]:
def save_code_to_file(text: str, filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

In [12]:
mojo_code = """
from gpu import thread_idx, block_idx, block_dim, lane_id
from gpu.host import DeviceContext
from gpu.warp import shuffle_xor, prefix_sum, WARP_SIZE
from layout import Layout, LayoutTensor
from sys import argv
from testing import assert_equal, assert_almost_equal

# ANCHOR: butterfly_pair_swap
alias SIZE = WARP_SIZE
alias BLOCKS_PER_GRID = (1, 1)
alias THREADS_PER_BLOCK = (WARP_SIZE, 1)
alias dtype = DType.float32
alias layout = Layout.row_major(SIZE)


fn butterfly_pair_swap[
    layout: Layout, size: Int
](
    output: LayoutTensor[mut=False, dtype, layout],
    input: LayoutTensor[mut=False, dtype, layout],
):
    global_i = block_dim.x * block_idx.x + thread_idx.x

    # FILL ME IN (4 lines)
    if global_i < size:
        current_val = input[global_i]
        swapped_val = shuffle_xor(current_val, 1)
        output[global_i] = swapped_val

# ANCHOR_END: butterfly_pair_swap


# ANCHOR: butterfly_parallel_max
fn butterfly_parallel_max[
    layout: Layout, size: Int
](
    output: LayoutTensor[mut=False, dtype, layout],
    input: LayoutTensor[mut=False, dtype, layout],
):
    global_i = block_dim.x * block_idx.x + thread_idx.x

    # FILL ME IN (roughly 7 lines)
    if global_i < size:
        max_val = input[global_i]
        offset = WARP_SIZE // 2
        while offset > 0:
            max_val = max(max_val, shuffle_xor(max_val, offset))
            offset //= 2

        output[global_i] = max_val

# ANCHOR_END: butterfly_parallel_max


# ANCHOR: butterfly_conditional_max
alias SIZE_2 = 64
alias BLOCKS_PER_GRID_2 = (2, 1)
alias THREADS_PER_BLOCK_2 = (WARP_SIZE, 1)
alias layout_2 = Layout.row_major(SIZE_2)


fn butterfly_conditional_max[
    layout: Layout, size: Int
](
    output: LayoutTensor[mut=False, dtype, layout],
    input: LayoutTensor[mut=False, dtype, layout],
):
    global_i = block_dim.x * block_idx.x + thread_idx.x
    lane = lane_id()

    if global_i < size:
        current_val = input[global_i]
        min_val = current_val

        # FILL ME IN (roughly 11 lines)
        offset = WARP_SIZE // 2
        while offset > 0:
            neighbor_val = shuffle_xor(current_val, offset)
            current_val = max(current_val, neighbor_val)

            min_neighbor_val = shuffle_xor(min_val, offset)
            min_val = min(min_val, min_neighbor_val)

            offset //= 2
        if lane % 2 == 0:
            output[global_i] = current_val
        else:
            output[global_i] = min_val


# ANCHOR_END: butterfly_conditional_max


# ANCHOR: warp_inclusive_prefix_sum
fn warp_inclusive_prefix_sum[
    layout: Layout, size: Int
](
    output: LayoutTensor[mut=False, dtype, layout],
    input: LayoutTensor[mut=False, dtype, layout],
):
    global_i = block_dim.x * block_idx.x + thread_idx.x

    # FILL ME IN (roughly 4 lines)
    if global_i < size:
        current_val = input[global_i]
        scan_result = prefix_sum[exclusive=False](
            rebind[Scalar[dtype]](current_val)
        )

        output[global_i] = scan_result



# ANCHOR_END: warp_inclusive_prefix_sum


# ANCHOR: warp_partition
fn warp_partition[
    layout: Layout, size: Int
](
    output: LayoutTensor[mut=False, dtype, layout],
    input: LayoutTensor[mut=False, dtype, layout],
    pivot: Float32,
):
    global_i = block_dim.x * block_idx.x + thread_idx.x

    if global_i < size:
        current_val = input[global_i]

        # FILL ME IN (roughly 13 lines)
        if global_i < size:
            current_val = input[global_i]
            predicate_left = Float32(1.0) if current_val < pivot else Float32(0.0)
            predicate_right = Float32(1.0) if current_val >= pivot else Float32(0.0)

            warp_left_pos = prefix_sum[exclusive=True](predicate_left)
            warp_right_pos = prefix_sum[exclusive=True](predicate_right)

            warp_left_total = predicate_left

            offset = WARP_SIZE // 2
            while offset > 0:
                warp_left_total += shuffle_xor(warp_left_total, offset)
                offset //= 2

            if current_val < pivot:
                output[Int(warp_left_pos)] = current_val
            else:
                output[Int(warp_left_total + warp_right_pos)] = current_val


# ANCHOR_END: warp_partition


def test_butterfly_pair_swap():
    with DeviceContext() as ctx:
        input_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
        output_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

        with input_buf.map_to_host() as input_host:
            for i in range(SIZE):
                input_host[i] = i

        input_tensor = LayoutTensor[mut=False, dtype, layout](
            input_buf.unsafe_ptr()
        )
        output_tensor = LayoutTensor[mut=False, dtype, layout](
            output_buf.unsafe_ptr()
        )

        ctx.enqueue_function[butterfly_pair_swap[layout, SIZE]](
            output_tensor,
            input_tensor,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        expected_buf = ctx.enqueue_create_host_buffer[dtype](SIZE).enqueue_fill(
            0
        )
        ctx.synchronize()

        # Create expected results: pairs should be swapped
        # (0,1) -> (1,0), (2,3) -> (3,2), (4,5) -> (5,4), etc.
        for i in range(SIZE):
            if i % 2 == 0:
                # Even positions get odd values
                expected_buf[i] = i + 1
            else:
                # Odd positions get even values
                expected_buf[i] = i - 1

        with output_buf.map_to_host() as output_host:
            print("output:", output_host)
            print("expected:", expected_buf)
            for i in range(SIZE):
                assert_equal(output_host[i], expected_buf[i])

    print("✅ Butterfly pair swap test passed!")


def test_butterfly_parallel_max():
    with DeviceContext() as ctx:
        input_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
        output_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

        with input_buf.map_to_host() as input_host:
            for i in range(SIZE):
                input_host[i] = i * 2
            # Make sure we have a clear maximum
            input_host[SIZE - 1] = 1000.0

        input_tensor = LayoutTensor[mut=False, dtype, layout](
            input_buf.unsafe_ptr()
        )
        output_tensor = LayoutTensor[mut=False, dtype, layout](
            output_buf.unsafe_ptr()
        )

        ctx.enqueue_function[butterfly_parallel_max[layout, SIZE]](
            output_tensor,
            input_tensor,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        ctx.synchronize()

        expected_buf = ctx.enqueue_create_host_buffer[dtype](SIZE).enqueue_fill(
            1000.0
        )

        # All threads should have the maximum value (1000.0)
        with output_buf.map_to_host() as output_host:
            print("output:", output_host)
            print("expected:", expected_buf)

            for i in range(SIZE):
                assert_almost_equal(output_host[i], 1000.0, rtol=1e-5)

    print("✅ Butterfly parallel max test passed!")


def test_butterfly_conditional_max():
    with DeviceContext() as ctx:
        input_buf = ctx.enqueue_create_buffer[dtype](SIZE_2).enqueue_fill(0)
        output_buf = ctx.enqueue_create_buffer[dtype](SIZE_2).enqueue_fill(0)

        with input_buf.map_to_host() as input_host:
            for i in range(SIZE_2):
                if i < 9:
                    values = [3, 1, 7, 2, 9, 4, 8, 5, 6]
                    input_host[i] = values[i]
                else:
                    input_host[i] = i % 10

        input_tensor = LayoutTensor[mut=False, dtype, layout_2](
            input_buf.unsafe_ptr()
        )
        output_tensor = LayoutTensor[mut=False, dtype, layout_2](
            output_buf.unsafe_ptr()
        )

        ctx.enqueue_function[butterfly_conditional_max[layout_2, SIZE_2]](
            output_tensor,
            input_tensor,
            grid_dim=BLOCKS_PER_GRID_2,
            block_dim=THREADS_PER_BLOCK_2,
        )

        ctx.synchronize()

        expected_buf = ctx.enqueue_create_host_buffer[dtype](
            SIZE_2
        ).enqueue_fill(0)

        # Expected: even lanes get max, odd lanes get min
        with input_buf.map_to_host() as input_host:
            max_val = input_host[0]
            min_val = input_host[0]
            for i in range(1, SIZE_2):
                if input_host[i] > max_val:
                    max_val = input_host[i]
                if input_host[i] < min_val:
                    min_val = input_host[i]

            for i in range(SIZE_2):
                if i % 2 == 0:
                    expected_buf[i] = max_val
                else:
                    expected_buf[i] = min_val

        with output_buf.map_to_host() as output_host:
            print("output:", output_host)
            print("expected:", expected_buf)

            for i in range(SIZE_2):
                if i % 2 == 0:
                    assert_almost_equal(output_host[i], max_val, rtol=1e-5)
                else:
                    assert_almost_equal(output_host[i], min_val, rtol=1e-5)

    print("✅ Butterfly conditional max test passed!")


def test_warp_inclusive_prefix_sum():
    with DeviceContext() as ctx:
        input_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
        output_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

        with input_buf.map_to_host() as input_host:
            for i in range(SIZE):
                input_host[i] = i + 1

        input_tensor = LayoutTensor[mut=False, dtype, layout](
            input_buf.unsafe_ptr()
        )
        output_tensor = LayoutTensor[mut=False, dtype, layout](
            output_buf.unsafe_ptr()
        )

        ctx.enqueue_function[warp_inclusive_prefix_sum[layout, SIZE]](
            output_tensor,
            input_tensor,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        expected_buf = ctx.enqueue_create_host_buffer[dtype](SIZE).enqueue_fill(
            0
        )
        ctx.synchronize()

        # Create expected inclusive prefix sum: [1, 3, 6, 10, 15, 21, 28, 36, ...]
        with input_buf.map_to_host() as input_host:
            expected_buf[0] = input_host[0]
            for i in range(1, SIZE):
                expected_buf[i] = expected_buf[i - 1] + input_host[i]

        with output_buf.map_to_host() as output_host:
            print("output:", output_host)
            print("expected:", expected_buf)
            for i in range(SIZE):
                assert_almost_equal(output_host[i], expected_buf[i], rtol=1e-5)

    print("✅ Warp inclusive prefix sum test passed!")


def test_warp_partition():
    with DeviceContext() as ctx:
        input_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
        output_buf = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

        # Create test data: mix of values above and below pivot
        pivot_value = Float32(5.0)
        with input_buf.map_to_host() as input_host:
            # Create: [3, 7, 1, 8, 2, 9, 4, 6, ...]
            test_values = [3, 7, 1, 8, 2, 9, 4, 6, 0, 10, 3, 11, 1, 12, 4, 13]
            for i in range(SIZE):
                input_host[i] = test_values[i % len(test_values)]

        input_tensor = LayoutTensor[mut=False, dtype, layout](
            input_buf.unsafe_ptr()
        )
        output_tensor = LayoutTensor[mut=False, dtype, layout](
            output_buf.unsafe_ptr()
        )

        ctx.enqueue_function[warp_partition[layout, SIZE]](
            output_tensor,
            input_tensor,
            pivot_value,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        expected_buf = ctx.enqueue_create_host_buffer[dtype](SIZE).enqueue_fill(
            0
        )
        ctx.synchronize()

        # Create expected results: elements < 5 on left, >= 5 on right
        with input_buf.map_to_host() as input_host:
            left_values = List[Float32]()
            right_values = List[Float32]()

            for i in range(SIZE):
                if input_host[i] < pivot_value:
                    left_values.append(input_host[i])
                else:
                    right_values.append(input_host[i])

            # Fill expected buffer
            for i in range(len(left_values)):
                expected_buf[i] = left_values[i]
            for i in range(len(right_values)):
                expected_buf[len(left_values) + i] = right_values[i]

        with output_buf.map_to_host() as output_host:
            print("output:", output_host)
            print("expected:", expected_buf)
            print("pivot:", pivot_value)

            # Verify partitioning property (left < pivot, right >= pivot)
            # Find partition boundary
            var partition_point = 0
            for i in range(SIZE):
                if output_host[i] >= pivot_value:
                    partition_point = i
                    break

            # Check left partition
            for i in range(partition_point):
                if output_host[i] >= pivot_value:
                    print("ERROR: Left partition contains value >= pivot")

            # Check right partition
            for i in range(partition_point, SIZE):
                if output_host[i] < pivot_value:
                    print("ERROR: Right partition contains value < pivot")

    print("✅ Warp partition test passed!")


def main():
    print("WARP_SIZE: ", WARP_SIZE)
    if len(argv()) < 2:
        print(
            "Usage: p24.mojo"
            " [--pair-swap|--parallel-max|--conditional-max|--prefix-sum|--partition]"
        )
        return

    test_type = argv()[1]
    if test_type == "--pair-swap":
        print("SIZE: ", SIZE)
        test_butterfly_pair_swap()
    elif test_type == "--parallel-max":
        print("SIZE: ", SIZE)
        test_butterfly_parallel_max()
    elif test_type == "--conditional-max":
        print("SIZE: ", SIZE_2)
        test_butterfly_conditional_max()
    elif test_type == "--prefix-sum":
        print("SIZE: ", SIZE)
        test_warp_inclusive_prefix_sum()
    elif test_type == "--partition":
        print("SIZE: ", SIZE)
        test_warp_partition()
    else:
        print(
            "Usage: p24.mojo"
            " [--pair-swap|--parallel-max|--conditional-max|--prefix-sum|--partition]"
        )
"""

In [13]:
save_code_to_file(mojo_code, "/content/mojo-gpu-puzzles/problems/p26/p26.mojo")

In [14]:
!cd /content/mojo-gpu-puzzles && uv run poe p26 --partition

Poe => mojo problems/p26/p26.mojo --prefix-sum
/content/mojo-gpu-puzzles/problems/p26/p26.mojo:129:28: warning: assignment to 'current_val' was never used; assign to '_' instead?
        current_val = input[global_i]
                           ^
WARP_SIZE:  32
SIZE:  32
output: HostBuffer([1.0, 3.0, 6.0, 10.0, 15.0, 21.0, 28.0, 36.0, 45.0, 55.0, 66.0, 78.0, 91.0, 105.0, 120.0, 136.0, 153.0, 171.0, 190.0, 210.0, 231.0, 253.0, 276.0, 300.0, 325.0, 351.0, 378.0, 406.0, 435.0, 465.0, 496.0, 528.0])
expected: HostBuffer([1.0, 3.0, 6.0, 10.0, 15.0, 21.0, 28.0, 36.0, 45.0, 55.0, 66.0, 78.0, 91.0, 105.0, 120.0, 136.0, 153.0, 171.0, 190.0, 210.0, 231.0, 253.0, 276.0, 300.0, 325.0, 351.0, 378.0, 406.0, 435.0, 465.0, 496.0, 528.0])
✅ Warp inclusive prefix sum test passed!
